# Boilerplate - inicialização padrão

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setAppName("pyspark-bolsa-familia")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# Caso de Uso: Análise Pagamentos Bolsa Família

#### Adicionando arquivo "202001-bolsa-familia-pgto-sample.csv" no HDFS

In [3]:
! hadoop fs -ls hdfs://node-master:9000/user/root

Found 4 items
drwxr-xr-x   - root supergroup          0 2023-03-07 21:17 hdfs://node-master:9000/user/root/.sparkStaging
-rw-r--r--   2 root supergroup    5604051 2023-03-07 21:13 hdfs://node-master:9000/user/root/202001-bolsa-familia-pgto-sample.csv
-rw-r--r--   2 root supergroup  606777344 2023-03-07 21:17 hdfs://node-master:9000/user/root/202001_BolsaFamilia_Pagamentos.csv._COPYING_
drwxr-xr-x   - root supergroup          0 2023-03-07 21:02 hdfs://node-master:9000/user/root/f1


In [4]:
! hadoop fs -put ../datasets/202001-bolsa-familia-pgto-sample.csv

put: `202001-bolsa-familia-pgto-sample.csv': File exists


#### Verificando a adição no HDFS

In [5]:
! hadoop fs -ls hdfs://node-master:9000/user/root

Found 4 items
drwxr-xr-x   - root supergroup          0 2023-03-07 21:17 hdfs://node-master:9000/user/root/.sparkStaging
-rw-r--r--   2 root supergroup    5604051 2023-03-07 21:13 hdfs://node-master:9000/user/root/202001-bolsa-familia-pgto-sample.csv
-rw-r--r--   2 root supergroup  606777344 2023-03-07 21:17 hdfs://node-master:9000/user/root/202001_BolsaFamilia_Pagamentos.csv._COPYING_
drwxr-xr-x   - root supergroup          0 2023-03-07 21:02 hdfs://node-master:9000/user/root/f1


In [6]:
! hdfs fsck hdfs://node-master:9000/user/root/202001-bolsa-familia-pgto-sample.csv -files -blocks

Connecting to namenode via http://node-master:50070/fsck?ugi=root&files=1&blocks=1&path=%2Fuser%2Froot%2F202001-bolsa-familia-pgto-sample.csv
FSCK started by root (auth:SIMPLE) from /172.18.0.4 for path /user/root/202001-bolsa-familia-pgto-sample.csv at Tue Mar 07 21:18:12 GMT 2023
/user/root/202001-bolsa-familia-pgto-sample.csv 5604051 bytes, 1 block(s):  OK
0. BP-1661134654-172.18.0.4-1678222199052:blk_1073741853_1029 len=5604051 repl=2

Status: HEALTHY
 Total size:	5604051 B
 Total dirs:	0
 Total files:	1
 Total symlinks:		0
 Total blocks (validated):	1 (avg. block size 5604051 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	2
 Average block replication:	2.0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)
 Number of data-nodes:		2
 Number of racks:		1
FSCK ended at Tue Mar 07 21:18:12 GMT 2023 in 0 milliseconds


The filesystem under path '/user/root/

#### Carregando DataFrame a partir de um caminho no HDFS

In [7]:
HDFS_PATH_BOLSA_FAMILIA = "hdfs://node-master:9000/user/root/202001-bolsa-familia-pgto-sample.csv"

#### Encoding errado => UTF-8

In [8]:
dfe = spark.read\
          .options(delimiter=";", header=True, encoding="utf-8")\
          .csv(HDFS_PATH_BOLSA_FAMILIA)

In [9]:
dfe.head()

Row(M�S COMPET�NCIA='202001', M�S REFER�NCIA='201901', UF='MG', C�DIGO MUNIC�PIO SIAFI='4123', NOME MUNIC�PIO='BELO HORIZONTE', CPF FAVORECIDO='***.361.206-**', NIS FAVORECIDO='12581466091', NOME FAVORECIDO='ADRIANA RANGEL SANSAO', VALOR PARCELA='253,00')

#### Encoding certo => ISO-8859-1

In [10]:
df = spark.read\
          .options(delimiter=";", header=True, encoding="iso-8859-1")\
          .csv(HDFS_PATH_BOLSA_FAMILIA)

In [11]:
df.head(10)

[Row(MÊS COMPETÊNCIA='202001', MÊS REFERÊNCIA='201901', UF='MG', CÓDIGO MUNICÍPIO SIAFI='4123', NOME MUNICÍPIO='BELO HORIZONTE', CPF FAVORECIDO='***.361.206-**', NIS FAVORECIDO='12581466091', NOME FAVORECIDO='ADRIANA RANGEL SANSAO', VALOR PARCELA='253,00'),
 Row(MÊS COMPETÊNCIA='202001', MÊS REFERÊNCIA='201901', UF='MG', CÓDIGO MUNICÍPIO SIAFI='4123', NOME MUNICÍPIO='BELO HORIZONTE', CPF FAVORECIDO='***.421.636-**', NIS FAVORECIDO='16057458312', NOME FAVORECIDO='ALEXSANDRA PEREIRA MUTZ', VALOR PARCELA='253,00'),
 Row(MÊS COMPETÊNCIA='202001', MÊS REFERÊNCIA='201901', UF='MG', CÓDIGO MUNICÍPIO SIAFI='4123', NOME MUNICÍPIO='BELO HORIZONTE', CPF FAVORECIDO='***.770.636-**', NIS FAVORECIDO='16127018278', NOME FAVORECIDO='ELIANA CRISTINA ROCHA SILVA', VALOR PARCELA='212,00'),
 Row(MÊS COMPETÊNCIA='202001', MÊS REFERÊNCIA='201901', UF='MG', CÓDIGO MUNICÍPIO SIAFI='4123', NOME MUNICÍPIO='BELO HORIZONTE', CPF FAVORECIDO='***.331.186-**', NIS FAVORECIDO='12135368026', NOME FAVORECIDO='ELIANE PE

#### Registrando DataFrame como Tabela

In [12]:
df.registerTempTable("beneficiario")

#### Consultando dados com o comando `spark.sql`

In [13]:
total = spark.sql("""
    select count(0) from beneficiario
""")

In [14]:
total

DataFrame[count(0): bigint]

In [15]:
total.show()

+--------+
|count(0)|
+--------+
|   49999|
+--------+



In [16]:
por_uf = spark.sql("""
    select UF, count(0) total
    from beneficiario
    group by UF
    order by total desc
""")

In [17]:
por_uf

DataFrame[UF: string, total: bigint]

In [18]:
por_uf.show(10)

+---+-----+
| UF|total|
+---+-----+
| SP| 7411|
| BA| 7356|
| RJ| 4085|
| PA| 3602|
| MA| 3440|
| CE| 3338|
| MG| 3239|
| AM| 2646|
| AL| 2389|
| PE| 2321|
+---+-----+
only showing top 10 rows



In [19]:
df.printSchema()

root
 |-- MÊS COMPETÊNCIA: string (nullable = true)
 |-- MÊS REFERÊNCIA: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: string (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: string (nullable = true)



#### Coluna VALOR PARCELA sendo parseada como string devido ao separador

In [20]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import FloatType

In [21]:
df = df.withColumn('VALOR', regexp_replace('VALOR PARCELA', '\\,', '\\.').cast("float"))

In [22]:
df.printSchema()

root
 |-- MÊS COMPETÊNCIA: string (nullable = true)
 |-- MÊS REFERÊNCIA: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: string (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: string (nullable = true)
 |-- VALOR: float (nullable = true)



In [23]:
df.head(1)

[Row(MÊS COMPETÊNCIA='202001', MÊS REFERÊNCIA='201901', UF='MG', CÓDIGO MUNICÍPIO SIAFI='4123', NOME MUNICÍPIO='BELO HORIZONTE', CPF FAVORECIDO='***.361.206-**', NIS FAVORECIDO='12581466091', NOME FAVORECIDO='ADRIANA RANGEL SANSAO', VALOR PARCELA='253,00', VALOR=253.0)]

In [24]:
df = df.withColumn('VALOR', df['VALOR'].cast("float"))

In [25]:
df.printSchema()

root
 |-- MÊS COMPETÊNCIA: string (nullable = true)
 |-- MÊS REFERÊNCIA: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: string (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: string (nullable = true)
 |-- VALOR: float (nullable = true)



In [26]:
df.head(1)

[Row(MÊS COMPETÊNCIA='202001', MÊS REFERÊNCIA='201901', UF='MG', CÓDIGO MUNICÍPIO SIAFI='4123', NOME MUNICÍPIO='BELO HORIZONTE', CPF FAVORECIDO='***.361.206-**', NIS FAVORECIDO='12581466091', NOME FAVORECIDO='ADRIANA RANGEL SANSAO', VALOR PARCELA='253,00', VALOR=253.0)]

In [27]:
df.registerTempTable("beneficiario")

In [28]:
dff = valor_pago_por_uf = spark.sql("""
    select UF, sum(VALOR) total
    from beneficiario
    group by UF
    order by total desc
""")

In [29]:
dff.show(10)

+---+---------+
| UF|    total|
+---+---------+
| SP|1202187.0|
| BA|1015792.0|
| RJ| 604660.0|
| PA| 522674.0|
| MG| 496694.0|
| MA| 487869.0|
| CE| 469233.0|
| AM| 463960.0|
| AL| 339074.0|
| PE| 321076.0|
+---+---------+
only showing top 10 rows



#### Mesmo processo, mas agora com o arquivo completo (1,6GB)

In [30]:
! hadoop fs -ls hdfs://node-master:9000/user/root

Found 4 items
drwxr-xr-x   - root supergroup          0 2023-03-07 21:17 hdfs://node-master:9000/user/root/.sparkStaging
-rw-r--r--   2 root supergroup    5604051 2023-03-07 21:13 hdfs://node-master:9000/user/root/202001-bolsa-familia-pgto-sample.csv
-rw-r--r--   2 root supergroup  606777344 2023-03-07 21:17 hdfs://node-master:9000/user/root/202001_BolsaFamilia_Pagamentos.csv._COPYING_
drwxr-xr-x   - root supergroup          0 2023-03-07 21:02 hdfs://node-master:9000/user/root/f1


In [31]:
%%time
! hadoop fs -put ../datasets/202001_BolsaFamilia_Pagamentos.csv

CPU times: user 1.49 s, sys: 549 ms, total: 2.04 s
Wall time: 37.5 s


In [32]:
! hdfs fsck hdfs://node-master:9000/user/root/202001_BolsaFamilia_Pagamentos.csv -files -blocks

Connecting to namenode via http://node-master:50070/fsck?ugi=root&files=1&blocks=1&path=%2Fuser%2Froot%2F202001_BolsaFamilia_Pagamentos.csv
FSCK started by root (auth:SIMPLE) from /172.18.0.4 for path /user/root/202001_BolsaFamilia_Pagamentos.csv at Tue Mar 07 21:19:09 GMT 2023
/user/root/202001_BolsaFamilia_Pagamentos.csv 1496845263 bytes, 12 block(s):  OK
0. BP-1661134654-172.18.0.4-1678222199052:blk_1073741864_1040 len=134217728 repl=2
1. BP-1661134654-172.18.0.4-1678222199052:blk_1073741865_1041 len=134217728 repl=2
2. BP-1661134654-172.18.0.4-1678222199052:blk_1073741866_1042 len=134217728 repl=2
3. BP-1661134654-172.18.0.4-1678222199052:blk_1073741867_1043 len=134217728 repl=2
4. BP-1661134654-172.18.0.4-1678222199052:blk_1073741868_1044 len=134217728 repl=2
5. BP-1661134654-172.18.0.4-1678222199052:blk_1073741869_1045 len=134217728 repl=2
6. BP-1661134654-172.18.0.4-1678222199052:blk_1073741870_1046 len=134217728 repl=2
7. BP-1661134654-172.18.0.4-1678222199052:blk_1073741871_10

In [33]:
HDFS_PATH_BOLSA_FAMILIA_FULL = "hdfs://node-master:9000/user/root/202001_BolsaFamilia_Pagamentos.csv"

In [34]:
df2 = spark.read\
          .options(delimiter=";", header=True, encoding="iso-8859-1")\
          .csv(HDFS_PATH_BOLSA_FAMILIA_FULL)

In [35]:
%%time
df2.show()

+---------------+--------------+---+----------------------+--------------+--------------+--------------+--------------------+-------------+
|MÊS COMPETÊNCIA|MÊS REFERÊNCIA| UF|CÓDIGO MUNICÍPIO SIAFI|NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|     NOME FAVORECIDO|VALOR PARCELA|
+---------------+--------------+---+----------------------+--------------+--------------+--------------+--------------------+-------------+
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.361.206-**|   12581466091|ADRIANA RANGEL SA...|       253,00|
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.421.636-**|   16057458312|ALEXSANDRA PEREIR...|       253,00|
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.770.636-**|   16127018278|ELIANA CRISTINA R...|       212,00|
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.331.186-**|   12135368026|      ELIANE PEREIRA|       212,00|
|         202001|   

In [36]:
df2.registerTempTable("beneficiario_full")

In [37]:
dff2 = spark.sql("""
    select count(0) from beneficiario_full
""")

In [38]:
%%time
dff2.show()

+--------+
|count(0)|
+--------+
|13301798|
+--------+

CPU times: user 2.89 ms, sys: 1.34 ms, total: 4.23 ms
Wall time: 10.2 s


In [39]:
por_uf_2 = spark.sql("""
    select UF, count(0) total
    from beneficiario_full
    group by UF
    order by total desc
""")

In [40]:
%%time
por_uf_2.show()

+---+-------+
| UF|  total|
+---+-------+
| BA|1756158|
| SP|1406214|
| PE|1129471|
| CE|1018022|
| MG| 988240|
| MA| 932253|
| PA| 928797|
| RJ| 827020|
| PB| 503919|
| PI| 438913|
| AM| 389239|
| AL| 389138|
| PR| 345636|
| RN| 340825|
| RS| 336812|
| GO| 286249|
| SE| 280256|
| ES| 170655|
| MT| 146210|
| MS| 115804|
+---+-------+
only showing top 20 rows

CPU times: user 4.36 ms, sys: 845 µs, total: 5.21 ms
Wall time: 23.3 s


In [41]:
df3 = df2.withColumn('VALOR', regexp_replace('VALOR PARCELA', '\\,', '\\.').cast("float"))

In [42]:
%%time
df3.printSchema()

root
 |-- MÊS COMPETÊNCIA: string (nullable = true)
 |-- MÊS REFERÊNCIA: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: string (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: string (nullable = true)
 |-- VALOR: float (nullable = true)

CPU times: user 2.66 ms, sys: 197 µs, total: 2.86 ms
Wall time: 4.32 ms


In [43]:
%%time
df3.show()

+---------------+--------------+---+----------------------+--------------+--------------+--------------+--------------------+-------------+-----+
|MÊS COMPETÊNCIA|MÊS REFERÊNCIA| UF|CÓDIGO MUNICÍPIO SIAFI|NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|     NOME FAVORECIDO|VALOR PARCELA|VALOR|
+---------------+--------------+---+----------------------+--------------+--------------+--------------+--------------------+-------------+-----+
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.361.206-**|   12581466091|ADRIANA RANGEL SA...|       253,00|253.0|
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.421.636-**|   16057458312|ALEXSANDRA PEREIR...|       253,00|253.0|
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.770.636-**|   16127018278|ELIANA CRISTINA R...|       212,00|212.0|
|         202001|        201901| MG|                  4123|BELO HORIZONTE|***.331.186-**|   12135368026|      ELIANE PEREIRA

In [44]:
df3.registerTempTable("beneficiario_full")

In [45]:
dff = valor_pago_por_uf = spark.sql("""
    select UF, sum(VALOR) total
    from beneficiario_full
    group by UF
    order by total desc
""")

In [46]:
dff.head()

Row(UF='BA', total=331509144.0)